In [79]:
from backend.dsm2_reader import read_scenario_dir
import pandas as pd
import altair as alt
from data_config import gatef, elev_list, flow_list, stn_name


In [80]:
stn_list = ['MID_GATEOP','GLC_GATEOP','OLD_GATEOP']

In [7]:
data = read_scenario_dir("C:/Users/Inigo/Projects/sdg-dashboard/data", v7_filter="_7")

mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV1Ma_SDG_7.dss
mathing:		C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_hydro_7.dss <----> C:\Users\Inigo\Projects\sdg-dashboard\data\output\FPV2Ma_SDG_7.dss
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.
DSS file opened successfully.


In [28]:
print(data.keys())

dict_keys(['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss'])


In [112]:
def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

def filter_data(data, column, values):
    """Filter data based on a column and a list of values."""
    return data[data[column].isin(values)].dropna()

def rename_gates(data, column, rename_map):
    """Rename values in a column in the DataFrame."""
    data[column] = data[column].replace(rename_map)
    return data

# def filter_date(data, column, start_date, end_date):
#     return data[column][start_date:end_date]

# def get_gate_up(data, column)

def prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef):
    """
    Prepare gate data dictionary from input datasets.
    
    Parameters:
    - sdg_flow: DataFrame with flow data.
    - sdg_stage: DataFrame with stage data.
    - gatop: DataFrame with gate operation data.
    - hydro_wl: Dataframe with water level data.
    - gatef: DataFrame containing gate configuration.

    Returns:
    - Dictionary containing gate data.
    """
    full_data = {}
    for i, gate_name in enumerate(gatef['name']):
        flow_data = filter_data(sdg_flow, 'gate_op', [gatef["flow_op"][i]])
        gate_data = filter_data(sdg_stage, 'gate_op', [gatef['gate_status'][i]])
        gate_data = gate_data[gate_data['unit'] == "FEET"]
        gateop_data = sdg_gateop[sdg_gateop['gate_op']==gatef['station'][i]]
        hydro_wl_data = hydro_wl[hydro_wl['gate']==gatef['station'][i]]
        full_data[gatef["ID"][i]] = {
            'name': gatef['name'][i],
            'bottom_elev': gatef['bottom_elev'][i],
            'width': gatef['width'][i],
            'flow_data': flow_data, #sc2_flow
            'gate_data': gate_data, #sc2_gate
            'gate_operation_data': gateop_data, #sc2_gateop
            "water_level_data": hydro_wl_data #sc2_wl
            
        }
    return full_data

def generate_full_data(data, path, gatef, elev_list, flow_list, stn_name, stn_list):
    sdg = data[path]['sdg']
    hydro = data[path]['hydro']

    sdg_stage = filter_data(sdg, 'gate_op', elev_list)
    sdg_flow = filter_data(sdg, 'gate_op', flow_list)
    sdg_gateop = filter_data(sdg, 'gate_op', stn_list)
    gate_names = {'MID_GATEOP': 'MHO', 'GLC_GATEOP': 'DGL', 'OLD_GATEOP': 'OLD'}
    sdg_gateop = rename_gates(sdg_gateop, "gate_op", gate_names)
    
    hydro_wl = hydro[hydro['parameter']=="STAGE"]
    hydro_wl = filter_data(hydro_wl, 'gate', stn_name)

    full_data = prepare_full_data(sdg_flow, sdg_stage, sdg_gateop, hydro_wl, gatef)
    sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
    sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']['value']
    glc_bottom_elev = full_data['GLC']['bottom_elev']
    glc_width = full_data['GLC']['width']

    sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
    sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
    mid_bottom_elev = full_data['MID']['bottom_elev']
    mid_width = full_data['MID']['width']
    
    sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
    sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
    old_bottom_elev = full_data['OLD']['bottom_elev']
    old_width = full_data['OLD']['width']

    full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
    full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
    full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

    return full_data


In [113]:
fpv1ma_hydro_full_data = generate_full_data(data, 
                 'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss', 
                 gatef,
                 elev_list,
                 flow_list,
                 stn_name,
                 stn_list
                 )

fpv2ma_hydro_full_data = generate_full_data(data,
                              'C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV2Ma_hydro_7.dss',
                              gatef,
                              elev_list,
                              flow_list,
                              stn_name,
                              stn_list)

In [122]:
fpv2ma_hydro_full_data['GLC']['gate_operation_data']
gate_up = (fpv2ma_hydro_full_data['GLC']['gate_operation_data'])
print(gate_up)

                  datetime  value  unit gate_op scenario
95     2016-01-01 00:00:00   -6.0  FEET     DGL   FPV2MA
96     2016-01-01 00:15:00   -6.0  FEET     DGL   FPV2MA
97     2016-01-01 00:30:00   -6.0  FEET     DGL   FPV2MA
98     2016-01-01 00:45:00   -6.0  FEET     DGL   FPV2MA
99     2016-01-01 01:00:00   -6.0  FEET     DGL   FPV2MA
...                    ...    ...   ...     ...      ...
280602 2023-12-31 22:45:00   -6.0  FEET     DGL   FPV2MA
280603 2023-12-31 23:00:00   -6.0  FEET     DGL   FPV2MA
280604 2023-12-31 23:15:00   -6.0  FEET     DGL   FPV2MA
280605 2023-12-31 23:30:00   -6.0  FEET     DGL   FPV2MA
280606 2023-12-31 23:45:00   -6.0  FEET     DGL   FPV2MA

[280512 rows x 5 columns]


In [91]:
sdg = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['sdg']
sdg_stage = sdg[sdg['gate_op'].isin(elev_list)]
sdg_flow = sdg[sdg['gate_op'].isin(flow_list)]
sdg_gateop = sdg[sdg['gate_op'].isin(stn_list)]
gate_names = {'MID_GATEOP':'MHO',
              'GLC_GATEOP':'DGL',
              'OLD_GATEOP':'OLD'}
sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)
print(sdg_gateop.head())

hydro = data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss']['hydro']
hydro_wl = hydro[hydro['parameter']=="STAGE"]
hydro_wl = hydro_wl[hydro_wl['gate'].isin(stn_name)]
print(hydro_wl.head())

# print(wl.head())
# print(data['C:\\Users\\Inigo\\Projects\\sdg-dashboard\\data\\output\\FPV1Ma_hydro_7.dss'].keys())
# # hydro_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.
# print(hydro_gateop['gate'].unique())
# print(hydro_gateop[hydro_gateop['gate']=="DGL"])

C:\Users\Inigo\AppData\Local\Temp\ipykernel_53388\234552756.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdg_gateop['gate_op'] = sdg_gateop['gate_op'].replace(gate_names)


             datetime  value  unit gate_op scenario
0 2015-12-31 00:15:00    NaN  FEET     DGL   FPV1MA
1 2015-12-31 00:30:00    NaN  FEET     DGL   FPV1MA
2 2015-12-31 00:45:00    NaN  FEET     DGL   FPV1MA
3 2015-12-31 01:00:00    NaN  FEET     DGL   FPV1MA
4 2015-12-31 01:15:00    NaN  FEET     DGL   FPV1MA
             datetime  value  unit gate parameter scenario
0 2015-12-31 00:15:00    NaN  FEET  DGL     STAGE   FPV1MA
1 2015-12-31 00:30:00    NaN  FEET  DGL     STAGE   FPV1MA
2 2015-12-31 00:45:00    NaN  FEET  DGL     STAGE   FPV1MA
3 2015-12-31 01:00:00    NaN  FEET  DGL     STAGE   FPV1MA
4 2015-12-31 01:15:00    NaN  FEET  DGL     STAGE   FPV1MA


In [5]:
full_data = {}

for i, gate_name in enumerate(gatef['name']):

    flow_data = sdg_flow[sdg_flow.gate_op == gatef["flow_op"][i]]
    gate_data = sdg_stage[sdg_stage.gate_op== gatef['gate_status'][i]]
     
    # Drop rows with NaN values from flow_data and gate_data
    flow_data = flow_data.dropna()
    gate_data = gate_data.dropna()
    
    full_data[gatef["ID"][i]] = {
        'name': gatef['name'][i],
        'bottom_elev': gatef['bottom_elev'][i],
        'width': gatef['width'][i],
        'flow_data': flow_data,
        'gate_data': gate_data
    }
sdg_flow_GLC_FLOW_FISH = full_data['GLC']['flow_data']['value']
sdg_flow_GLC_GATE_UP = full_data['GLC']['gate_data']['value']
glc_bottom_elev = full_data['GLC']['bottom_elev']
glc_width = full_data['GLC']['width']



In [6]:
sdg_flow_MID_FLOW_FISH = full_data['MID']['flow_data']['value']
sdg_flow_MID_GATE_UP = full_data['MID']['gate_data']['value']
mid_bottom_elev = full_data['MID']['bottom_elev']
mid_width = full_data['MID']['width']

sdg_flow_OLD_FLOW_FISH = full_data['OLD']['flow_data']['value']
sdg_flow_OLD_GATE_UP = full_data['OLD']['gate_data']['value']
old_bottom_elev = full_data['OLD']['bottom_elev']
old_width = full_data['OLD']['width']

In [7]:
thr_vel = 8
# mask = (sdg_flow.index.month>=5) & (sdg_flow.index.month<12) # time when gate are in operation
# vel=pd.DataFrame()
# print(sdg_flow['scenario'].unique())
full_data["GLC"]['vel'] = calc_vel(sdg_flow_GLC_FLOW_FISH,sdg_flow_GLC_GATE_UP,glc_bottom_elev, glc_width)
full_data["MID"]['vel'] = calc_vel(sdg_flow_MID_FLOW_FISH,sdg_flow_MID_GATE_UP,mid_bottom_elev, mid_width)
full_data["OLD"]['vel'] = calc_vel(sdg_flow_OLD_FLOW_FISH,sdg_flow_OLD_GATE_UP,old_bottom_elev, old_width)

# vel['MID'] = calc_vel(sdg_flow.MID_FLOW_FISH,sdg_flow.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
# vel['OLD'] = calc_vel(sdg_flow.OLD_FLOW_FISH,sdg_flow.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )
# print(full_data["GLC"]['vel'].tail())
print(full_data.keys())

dict_keys(['GLC', 'MID', 'OLD'])
